[Open Notebook here](https://colab.research.google.com/github/Vampire-Chan/SDUI/blob/notebook/ipynb)


In [ ]:
#@title **Environment Setup**
#@markdown Let ComfyUI Update and setup automatically for you. Untick if you don't use Google Drive.

# Import necessary modules
import os  # Required for checking file paths
from IPython.display import clear_output  # Optional, to clear output after drive mounting

# User-configurable options with checkboxes for simple control
OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}

# Store options in the dictionary
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

# Step 1: Mount Google Drive (if selected)
if OPTIONS['USE_GOOGLE_DRIVE']:
    print("\u2714 Mounting Google Drive...")  # Checkmark for success message

    %cd /
    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
    %cd /content/drive/MyDrive
else:
    WORKSPACE = 'ComfyUI'
    CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"

# Step 2: Clone the ComfyUI repository (if not already present)
if not os.path.exists(WORKSPACE):
    print("\u2714 Initial setup for ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE
else:
    print("\u2714 ComfyUI folder already exists, skipping clone.")

%cd $WORKSPACE

# Step 3: Update the ComfyUI repository (if selected)
if OPTIONS['UPDATE_COMFY_UI']:
    print("\u2714 Updating ComfyUI...")
    !git pull
else:
    print("\u2714 Skipping ComfyUI update.")

# Step 4: Clone or Update the ComfyUI-Manager repository
if not os.path.exists(CUSTOM_NODES_DIR):
    print("\u2714 Cloning ComfyUI-Manager for the first time...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git $CUSTOM_NODES_DIR
else:
    %cd $CUSTOM_NODES_DIR
    if OPTIONS['UPDATE_COMFY_UI']:
        print("\u2714 Updating ComfyUI-Manager...")
        !git pull
    else:
        print("\u2714 Skipping ComfyUI-Manager update.")
    %cd $WORKSPACE

# Step 5: Install required dependencies
print("\u2714 Installing dependencies...")
!pip install xformers!=0.0.18 -r requirements.txt \
    --extra-index-url https://download.pytorch.org/whl/cu121 \
    --extra-index-url https://download.pytorch.org/whl/cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu117

print("\u2714 Environment setup complete.")


✔ Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
✔ ComfyUI folder already exists, skipping clone.
/content/drive/MyDrive/ComfyUI
✔ Updating ComfyUI...
Already up to date.
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
✔ Updating ComfyUI-Manager...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 37 (delta 25), reused 31 (delta 20), pack-reused 0 (from 0)
Unpacking objects: 100% (37/37), 314.59 KiB | 289.00 KiB/s, done.
From https://github.com/ltdrdata/ComfyUI-Manager
   75047ed..82933e2  main       -> origin/main
Updating 75047ed..82933e2
Fast-forward
 custom-node-list.json                |  125 +-
 extension-node-map.json              |  245 ++-
 github-stats.json                    | 4369 +++++++++++++++++++++++++++---------------------------
 

In [ ]:
#@title Download Models, Checkpoints, LoRAs and more...
#@markdown Download whatever models you want to download and save. Easy to use UI access. Again Untick if you don't use Drive.
import os
import requests
import shutil
import urllib.parse
import time

# 🔥 Custom Colab Form Inputs
MODEL_URL = "https://civitai.com/api/download/models/728705?type=Model&format=SafeTensor&size=full&fp=fp16"  #@param {type:"string"}
MODEL_TYPE = "checkpoints"  #@param ["checkpoints", "clip_vision", "vae", "loras", "controlnet", "style_models", "upscale_models"]
SAVE_TO_DRIVE = True  #@param {type:"boolean"}

# Define the paths for model directories
MODEL_DIRS = {
    'checkpoints': './models/checkpoints/',
    'clip_vision': './models/clip_vision/',
    'vae': './models/vae/',
    'loras': './models/loras/',
    'controlnet': './models/controlnet/',
    'style_models': './models/style_models/',
    'upscale_models': './models/upscale_models/'
}

# Create directories if they do not exist
for path in MODEL_DIRS.values():
    os.makedirs(path, exist_ok=True)

def download_file(url, save_path):
    """Download a file from a given URL and save it to the specified path, tracking size, progress, and speed."""
    try:
        # Send GET request to the URL
        response = requests.get(url, stream=True, allow_redirects=True)
        response.raise_for_status()

        # Extract filename from URL
        filename = os.path.basename(urllib.parse.urlparse(url).path)

        # If content-disposition header exists, override the filename
        if 'content-disposition' in response.headers:
            import cgi
            _, params = cgi.parse_header(response.headers['content-disposition'])
            filename = params.get('filename', filename)

        # Ensure the save path is valid
        save_path = os.path.join(save_path, filename)

        # Get total file size for progress tracking
        total_size = int(response.headers.get('Content-Length', 0))
        downloaded = 0
        start_time = time.time()

        with open(save_path, 'wb') as file:
            # Download the file in chunks and update progress
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded += len(chunk)

                    # Calculate download progress
                    percent = (downloaded / total_size) * 100

                    # Calculate download speed (MB/s)
                    elapsed_time = time.time() - start_time
                    speed = downloaded / (1024 * 1024 * elapsed_time)  # Speed in MB/s

                    # Print progress
                    print(f"\rDownloading {filename} - {percent:.2f}% - {speed:.2f} MB/s", end="")

        print(f"\n✅ Downloaded: {filename} to {save_path} - Total Size: {total_size / (1024 * 1024):.2f} MB")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error downloading {url}: {e}")

def setup_environment():
    """Setup Google Drive if selected."""
    if SAVE_TO_DRIVE:
        print("🔗 Mounting Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        WORKSPACE = '/content/drive/MyDrive/ComfyUI'
    else:
        WORKSPACE = './ComfyUI'

# 🌐 Start setup environment
setup_environment()

# 🌐 Start model download
if MODEL_URL:
    print(f"⏳ Starting download for {MODEL_URL} to the '{MODEL_TYPE}' directory...")
    save_path = MODEL_DIRS.get(MODEL_TYPE, './models/')
    download_file(MODEL_URL, save_path)
else:
    print("❌ No URL provided for download.")


🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Starting download for https://civitai.com/api/download/models/1085338?type=Model&format=SafeTensor&size=pruned&fp=fp16 to the 'checkpoints' directory...
✅ Downloaded: waiANINSFWPONYXL_v11.safetensors to ./models/checkpoints/waiANINSFWPONYXL_v11.safetensors - Total Size: 6616.63 MB


In [ ]:
#@title **ComfyUI**
#@markdown Launch the ComfyUI and enjoy Stable Diffusion Generation on Colab. Make sure to UnTick if you use Colab without Google Drive. And also if ComfyUI fails to load and shows "module not found" then put the module name in the "required_modules" box in the format.
mount_gdrive = True #@param {type:"boolean"}

import subprocess
import threading
import time
import socket
import os
from google.colab import drive

# Function to mount Google Drive if the user selects it
def mount_google_drive():
    """Mount Google Drive to the Colab environment."""
    if mount_gdrive:
        print("🔧 Mounting Google Drive...")
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted successfully!")
        except Exception as e:
            print(f"❌ Error mounting Google Drive: {e}")
            exit(1)
    else:
        print("🔧 Not mounting Google Drive, using local folder.")

# Function to start cloudflared download
def download_cloudflared():
    """Download cloudflared with progress indication."""
    try:
        print("⏳ Downloading cloudflared...")
        url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
        result = os.system(f"wget --progress=bar:force {url} -O cloudflared-linux-amd64.deb")
        if result != 0:
            raise Exception("❌ Error downloading cloudflared. Download failed.")
        print("\n✅ Download complete.")
    except Exception as e:
        print(str(e))
        exit(1)

# Function to install cloudflared
def install_cloudflared():
    """Install cloudflared .deb file."""
    try:
        print("🔧 Installing cloudflared...")
        result = os.system("dpkg -i cloudflared-linux-amd64.deb")
        if result != 0:
            raise Exception("❌ Error installing cloudflared. Installation failed.")
        print("✅ Installation complete.")
    except Exception as e:
        print(str(e))
        exit(1)

# Function to check if CUDA is available
def check_cuda_availability():
    """Check if CUDA is available and print message."""
    import torch
    if torch.cuda.is_available():
        print("✅ CUDA is available, using GPU.")
    else:
        print("❌ No CUDA device found. Running on CPU mode.")

# Function to monitor ComfyUI and launch cloudflared
def iframe_thread(port):
    """Monitor ComfyUI port and launch cloudflared when ready."""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')

# Set the folder path based on Google Drive mount option
folder_path = '/content/ComfyUI'  # Default folder path
mount_google_drive()  # Mount Google Drive if selected
if mount_gdrive:
    folder_path = '/content/drive/MyDrive/ComfyUI'  # Use Google Drive folder for ComfyUI

# Check for CUDA availability
check_cuda_availability()

# Automatically install missing modules if ModuleNotFoundError is raised
def install_missing_module(module_name):
    try:
        __import__(module_name)
    except ModuleNotFoundError:
        print(f"❌ Module '{module_name}' not found. Installing it now...")
        os.system(f"pip install {module_name}")
        print(f"✅ Module '{module_name}' installed successfully.")

# List of critical modules to check and install if missing
required_modules = ['torch', 'flask', 'flask_cors', 'requests', 'torchsde', 'spandrel', 'kornia', 'piexif', 'segment_anything']  #@param

# Check and install missing modules
for module in required_modules:
    install_missing_module(module)

# Start cloudflared download and installation
download_cloudflared()
install_cloudflared()

# Start the ComfyUI thread and monitor port
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Start ComfyUI from the selected folder path
!python {folder_path}/main.py --dont-print-server


🔧 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!
✅ CUDA is available, using GPU.
❌ Module 'piexif' not found. Installing it now...
✅ Module 'piexif' installed successfully.
⏳ Downloading cloudflared...

✅ Download complete.
🔧 Installing cloudflared...
✅ Installation complete.
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2024-12-15 07:17:30.272645
** Platform: Linux
** Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/drive/MyDrive/ComfyUI
** Log path: /content/drive/MyDrive/ComfyUI/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/rgthree-comfy
   6.8 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager

Total VRAM 1510

In [2]:
#@title **Compress Archive**

#@markdown Use this cell for zipping and deleting files inside the output folder so that you won't have to download everything again and again. It also serves as a backup in case Google Drive fails to read the contents written by Google Colab. This helps ensure you have a compressed archive of your files for safekeeping and efficient storage.

from google.colab import drive
drive.mount('/content/drive')
import os
import shutil
import zipfile

#@title Parameters
output_folder = "/content/drive/MyDrive/ComfyUI/output"  #@param {type: "string"}
zip_name = "output.zip"  #@param {type: "string"}
zip_destination = "/content/drive/MyDrive/"  #@param {type: "string"}

def zip_and_clean_output(output_folder, zip_name, zip_destination):
    """
    Zips the contents of the output folder, moves the zip file to the
    destination, and then deletes the original contents of the output folder.

    Args:
      output_folder: Path to the folder to be zipped.
      zip_name: Name of the resulting zip file.
      zip_destination: Path to where the zip file should be saved.
    """
    # Check if the folders exist
    if not os.path.exists(output_folder):
        print(f"Error: The folder '{output_folder}' does not exist.")
        return

    if not os.path.exists(zip_destination):
        print(f"Error: The destination folder '{zip_destination}' does not exist.")
        return

    # Get the initial size of the output folder
    total_original_size = sum(
        os.path.getsize(os.path.join(root, file))
        for root, _, files in os.walk(output_folder)
        for file in files
    )

    # Create the zip file path
    zip_file_path = os.path.join(zip_destination, zip_name)

    # Start zipping the contents of the output folder
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_folder)
                zipf.write(file_path, arcname)

    # Get the size of the zip file
    total_zipped_size = os.path.getsize(zip_file_path)

    # Calculate compression percentage
    if total_original_size > 0:
        compression_percentage = (1 - (total_zipped_size / total_original_size)) * 100
    else:
        compression_percentage = 0

    # Print compression statistics
    print(f"Original size: {total_original_size / 1024**2:.2f} MB")
    print(f"Zipped size: {total_zipped_size / 1024**2:.2f} MB")
    print(f"Compression percentage: {compression_percentage:.2f}%")

    # Delete the contents of the output folder (but not the folder itself)
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

    print(f"Successfully zipped the folder and deleted contents of {output_folder}")

# Run the function with the provided parameters
zip_and_clean_output(output_folder, zip_name, zip_destination)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Original size: 38.56 MB
Zipped size: 38.50 MB
Compression percentage: 0.17%
Successfully zipped the folder and deleted contents of /content/drive/MyDrive/ComfyUI/output
